# **Data Collection**

## **Data Collection of MindBodyGreen and Wikipedia**

## Import statements

In [1]:
from bs4 import BeautifulSoup
from collections import Counter, deque
import csv
import numpy as np
import os
import pandas as pd
import random
import re 
import requests
import string
import time
import ast  # Useful to convert a string in a list
from urllib.parse import urljoin, urlparse, unquote
from urllib3.exceptions import LocationParseError

In [52]:
nltk.download('punkt')
nltk.download('punkt_tab') # Required for new versions
nltk.download('wordnet')
nltk.download('omw-1.4')   # Required for the lemmatiser
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Constant definitions

In [2]:
# --- CONFIGURATION OF COLORS ---

GREEN = "\033[92m"
RED = "\033[91m"
BLUE = "\033[94m"
RESET = "\033[0m"

In [3]:
# --- CONFIGURATION OF HEADER ---

HEADER_BROWSER = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36 Edg/143.0.0.0",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/"}

session = requests.Session()
session.headers.update(HEADER_BROWSER)

In [4]:
# --- CONFIGURATION OF FILTERS ---

filters_noise = [
    # E-commerce
    "/product", "/products", "/shop", "/store", "/cart", "/checkout",
    # User
    "/account", "/login", "/register", "/profile", 
    # Legal and Administrative pages
    "/about", "/contact", "/privacy", "/terms", "/policy", "/legal", "/accessibility", 
    "/editorial-process", "/data-collection", "/disclaimer", "/cookies", "/sponsor",
    "/advertise", "/jobs", "/faq", "/help", "/wc", 
    # Site Structure
    "/tag", "/search", "/author",
    "mailto:", "tel:", "javascript:", ".jpg", ".png",
    "youtube.com", "instagram.com", "facebook.com", "twitter.com", "x.com",
    "pinterest.com", "linkedin.com", "tiktok.com", "amazon.com"
]

In [5]:
# --- CONFIGURATION OF SEED LINKS FOR WIKIPEDIA ---

wiki_seed_pages = [ # Our selection of a few pages = starting list
    "https://en.wikipedia.org/wiki/Lifestyle",
    "https://en.wikipedia.org/wiki/Lifestyle_trends_and_media",
    "https://en.wikipedia.org/wiki/Self-care",
    "https://en.wikipedia.org/wiki/Physical_fitness",
    "https://en.wikipedia.org/wiki/Healthy_diet",
    "https://en.wikipedia.org/wiki/Travel",
    "https://en.wikipedia.org/wiki/Outdoor_recreation",
    "https://en.wikipedia.org/wiki/Sustainable_living",
    "https://en.wikipedia.org/wiki/Fashion",
]

## Definitions implementation

In [6]:
def get_filenames(source_name): 
    # Create all files from a source using the name specified in the source cell [21]
    
    directory_name = "data"
    if not os.path.exists(directory_name): # Checks if the 'data' folder exists
        os.makedirs(directory_name) # Setup the output directory
        print(f"{GREEN}Created directory : {directory_name}{RESET}")
    
    # We assume that source_name contains no spaces. If changed, add : source_name = source_name.replace(" ", "_")

    return {"raw":       os.path.join(directory_name, f"{source_name}_raw_html.csv"),
            "nodes" :    os.path.join(directory_name, f"{source_name}_nodes.csv"),
            "edges" :    os.path.join(directory_name, f"{source_name}_edges.csv")}

In [7]:
def fetch_url(url) :
    # Fetches the content of a URL
    
    time.sleep(random.uniform(1, 3)) # Random delay between 1 and 3 seconds to avoid overloading the server

    try:
        response = session.get(url, timeout=10) # Timeout set to 10s to prevent the crawler from hanging on slow pages
        
        if response.status_code == 200: 
            return response # If the status code is not OK (200), the function returns none and an error message
        else:
            print(f"Failed to fetch the url: {url} with status code {response.status_code}")
            return None
    
    except LocationParseError: 
        return None

    except requests.RequestException: # Catch network-related errors (DNS failure, connection refused, etc.)
        return None

In [8]:
def to_soup(url): 
    # Parses the HTML content of a URL into a BeautifulSoup object
    
    response = fetch_url(url) 

    if response: # If the fetch was successful, parse the HTML
        return BeautifulSoup(response.text, 'html.parser') 
    else: # If the response is not none, the function return the beautiful soup object
        return None

In [9]:
def extract_links_breadth_first(url, max_levels=1): 
    # Crawls a website using Breadth-First Search (BFS)

    queue = deque([(url, 0)]) # Initialize BFS queue with (url, current_depth)
    visited_links = [url]  # Track visited URLs to avoid duplicates (list preserves order)
    last_level_links = [] # Store links found at the maximum level
    dico = {} # For the graph structure {parent : [child1, child2]}

    while queue:
        current_url, current_level = queue.popleft()

        if current_level == max_levels: # if the limit is reached, we store the links in the list
            if current_url not in last_level_links:
                last_level_links.append(current_url)
            continue
        elif current_level > max_levels:
            continue
    
        soup = to_soup(current_url) # Fetching and parsing
        if not soup:
            continue

        dico.setdefault(current_url, [])

        content_area = soup.body # Broad extraction scope: analysis of the entire <body> to retrieve as many links as possible
        if not content_area:
            continue

        links = sorted(content_area.find_all('a', href=True), key=lambda x: x.get('href')) # Extract all 'a' tags with an 'href' attribute and sort alphabetically
        
        for item in links :
            href = item.get('href') # href is short for Hypertext REFerence. It indicates the destination of the link
            full_url = urljoin(current_url, href)
            full_url = full_url.split('?')[0].split('#')[0] # Cut at the "?" of "#" and keep only the beginning : [0]
            full_url = full_url.lower()

            if any(filter in full_url for filter in filters_noise):
                continue
            
            dico[current_url].append(full_url) # Add edge to the adjacency list (Parent -> Child)

            if full_url not in visited_links: # If the link is new
                visited_links.append(full_url) 
                queue.append((full_url, current_level + 1)) # Add it to the queue for future crawling
                    
    return visited_links, last_level_links, dico

In [10]:
def extract_links_depth_first(start_url, visited_links, dico, max_depth=3, max_links_per_page=10, current_depth=0): 
    #Crawls a website using Depth-First Search (DFS)
    
    if current_depth >= max_depth:
        return

    soup = to_soup(start_url) # Fetching and parsing
    if not soup :
        return

    dico.setdefault(start_url, []) # Ensure the current URL exists in the adjacency list
    links_followed = 0 # Counter to max_links_per_page

    content_area = soup.body # Broad extraction scope: analysis of the entire <body> to retrieve as many links as possible
    if not content_area:
        return
    
    links = sorted(content_area.find_all('a', href=True), key=lambda x: x.get('href')) # Extract all 'a' tags with an 'href' attribute and sort alphabetically
    
    for link in links:
        if links_followed >= max_links_per_page: # Maximum number of links followed (crawled) from a given page
            break

        href = link.get('href')
        full_url = urljoin(start_url, href) # Convert relative paths to absolute URLs
        full_url = full_url.split('?')[0].split('#')[0] # Cut at the "?" of "#" and keep only the beginning : [0]
        full_url = full_url.lower()

        if any(filter in full_url for filter in filters_noise):
                continue

        dico[start_url].append(full_url) # Add edge to the adjacency list (Parent -> Child)

        if full_url not in visited_links: # If the link is new, explore it deeply before moving to the next one
            visited_links.append(full_url)
            links_followed += 1

            # Recursive call: dive deeper into this branch
            extract_links_depth_first( start_url=full_url, visited_links=visited_links, dico=dico, max_depth=max_depth, max_links_per_page=max_links_per_page, current_depth=current_depth + 1)
    
    return visited_links, dico


In [11]:
def extract_links_depth_from_list(max_links_to_dfs, start_links, visited_links, dico, max_depth=3, max_links_per_page=10):
    # Function to launch multiple DFS crawls from a list of seed URLs
    
    if visited_links is None:
        visited_links = []
    
    sorted_links = sorted(start_links) # Sorted because same input order = same results
    
    for link in sorted_links:
        if link not in visited_links:
            visited_links.append(link)

    for start_url in sorted_links[:max_links_to_dfs]: # Maximum number of seeds (starting points) that we want to crawl in depth  
        extract_links_depth_first(start_url=start_url, visited_links=visited_links, 
                                  dico=dico, max_depth=max_depth, 
                                  max_links_per_page=max_links_per_page, current_depth=0)

    return visited_links

In [12]:
def crawl_MBG_seed_links(url):
    # Definition of our parameters for Feedspots
    MAX_LINKS_TO_DFS = 30
    MAX_BREADTH_LEVEL = 1
    MAX_DEPTH = 2
    MAX_LINKS_PER_PAGE = 8 # Number of links per DFS

    # Breadth-First Search (wide exploration)
    breadth_links, last_level_links, dico = extract_links_breadth_first(url, max_levels=MAX_BREADTH_LEVEL)

    # Depth-First Search from the BFS frontier
    depth_links = extract_links_depth_from_list(max_links_to_dfs=MAX_LINKS_TO_DFS,
                                                start_links=last_level_links, 
                                                visited_links=breadth_links.copy(), 
                                                dico=dico, max_depth=MAX_DEPTH, 
                                                max_links_per_page=MAX_LINKS_PER_PAGE)

    all_links = set(breadth_links) | set(depth_links) # Union of BFS and DFS results (order is not preserved, but duplicates are removed)
    
    return list(all_links), dico

In [13]:
def extract_wikipedia_links(url):
    # Extracts internal Wikipedia article links from a given Wikipedia page
    
    wiki_links = set()

    soup = to_soup(url) 
    if not soup:
        return set() # Returns a list of URLs pointing to Wikipedia articles only

    # Focus on : Main content paragraphs (article body)
    main_content = soup.find_all('div', class_="mw-content-ltr mw-parser-output")
    for div in main_content:
        for p in div.find_all('p'): # Target: paragraphs (<p>) within the class (main content div)
            for item in p.find_all('a', href=True):
                href = item.get('href')
                if href.startswith("/wiki/") and ":" not in href: # Keep links "/wiki/..." and exclude pages containing ":"
                    wiki_links.add(urljoin(url, href))

    # Focus on : "See also" section (related articles)
    see_also = soup.find_all('div', class_='div-col')
    for div in see_also:
        for li in div.find_all('li'): # Target: lists (<li>) within columns ('div-col')
            for item in li.find_all('a', href=True):
                href = item.get('href')
                if href.startswith("/wiki/") and ":" not in href:
                    wiki_links.add(urljoin(url, href))

    return list(wiki_links)

In [14]:
def crawl_wiki_seed_links(wiki_seed_pages):
    
    wiki_all_links = set(wiki_seed_pages) # Set() avoids duplication, unlike lists
    dico_wiki_links = {}
    
    for url in wiki_seed_pages:
        crawl_links = extract_wikipedia_links(url)
        dico_wiki_links[url] = crawl_links
        wiki_all_links.update(crawl_links) # Only add new links

    return list(wiki_all_links), dico_wiki_links

In [15]:
def save_to_csv(data, filename):
    
    if not data:
        print(f"{RED}No data to save.{RESET}")
        return

    # Warn if the file already exists
    if os.path.exists(filename):
        print(f"{BLUE}Note: '{filename}' already exists and will be overwritten{RESET}")
 
    # Get the keys from the first dictionary for the CSV header from get_html_content
    try:
        fieldnames = data[0].keys() # Detection of the existing colons in the data file
    
        with open(filename, 'w', newline='', encoding='utf-8') as f: # Opens the csv file as utf-8 (= character encoding system)
            writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL) # Initialising a writer to write the dictionary into the csv file
            writer.writeheader() # Writes the colons headers
            writer.writerows(data) # Writes the rows

        print(f"{GREEN}Success! Data saved to: {RESET}{os.path.abspath(filename)}")
        
    except Exception as e:
        print(f"{RED}Error saving file: {e}{RESET}")

In [16]:
def get_html_content(links, output_csv, max=None):
    # Fetches raw HTML content for a list of URLs

    content = []
    failed_links = []

    for link in links[:max]:
        response = fetch_url(link)
        
        if response:
            content.append({'url': link, 'html': response.text}) # Store raw HTML as text for text extraction
        else:
            failed_links.append(link) # Keep track of failures for reporting

    print(f"{GREEN}Fetched: {len(content)}{RESET}")
    print(f"{RED}Failed: {len(failed_links)}{RESET}")

    if output_csv and content:
        save_to_csv(content, output_csv)

    return content, failed_links 

In [17]:
def create_nodes_csv(all_links, output_file):
    # Builds a 'nodes' CSV from an adjacency list (graph dictionary)

    nodes = set(all_links.keys())  # Each key of the dictionnary "all_links" is a node (set() don't keep any possible duplicate)
    
    for targets in all_links.values():
        nodes.update(targets)  # Each value of the dictionnary is a node

    with open(output_file, mode="w", newline="", encoding="utf-8") as f:  # Create (or overwrite) a csv file 
        writer = csv.writer(f)  # Create an object that will enable our code to write within the file
        writer.writerow(["node_id"])

        for node in nodes:
            writer.writerow([node])   

In [18]:
def create_edges_csv(links, output_file):
    # Builds an 'edges' CSV from an adjacency list (directed graph)
    
    with open(output_file, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["source", "target"])  # We start by writing the headers of our column

        for source, targets in links.items():
            for target in targets:  
                if target != source:  # Avoid self-loops (A -> A)
                    writer.writerow([source, target])  # If the target is different form the source (not A -> A), we add the relation in our file
                else :
                    continue

## Execution

###### For MindBodyGreen

In [19]:
MBG_source_name = "MBG"
MBG_start_point = "https://www.mindbodygreen.com/"

MBG_files = get_filenames(MBG_source_name) # Build all standardized output paths for this source 

In [20]:
print("\n=== CRAWLING ===")

print(f"{BLUE}[{MBG_source_name}] Starting the crawling...{RESET}")    
MBG_crawled_links, MBG_dico_links = crawl_MBG_seed_links(MBG_start_point)
print(f"{GREEN}-> Found {len(MBG_crawled_links)} links{RESET}")


=== CRAWLING ===
[MBG] Starting the crawling...
Failed to fetch the url: https://www.glassdoor.com/salaries/health-coach-salary-srch_ko0,12.htm with status code 403
Failed to fetch the url: https://www.google.com/url with status code 404
Failed to fetch the url: https://go.skimresources.com with status code 404
Failed to fetch the url: https://pubmed.ncbi.nlm.nih.gov/28329045/ with status code 403
Failed to fetch the url: https://pmc.ncbi.nlm.nih.gov/articles/pmc6279907/ with status code 403
Failed to fetch the url: https://pmc.ncbi.nlm.nih.gov/articles/pmc9524880/ with status code 403
Failed to fetch the url: https://pubmed.ncbi.nlm.nih.gov/35271050/ with status code 403
Failed to fetch the url: https://www.mindbodygreen.com/articles/how-to-master-pull-up-and-why-women-are-embracing-challenge with status code 404
Failed to fetch the url: https://pmc.ncbi.nlm.nih.gov/articles/pmc10777545/ with status code 403
Failed to fetch the url: https://pubmed.ncbi.nlm.nih.gov/19526369/ with stat

In [21]:
print("\n=== SCRAPING ===")

print(f"{BLUE}[{MBG_source_name}] Starting the scraping...{RESET}")
get_html_content((MBG_crawled_links), MBG_files['raw']) # Fetch HTML for each crawled URL and save it as a CSV (url, html)
print(f"{GREEN}Raw HTML data saved to {MBG_files['raw']}{RESET}")    


=== SCRAPING ===
[MBG] Starting the scraping...
Failed to fetch the url: https://www.cell.com/cell-reports-medicine/fulltext/s2666-3791(22)00474-8 with status code 403
Failed to fetch the url: https://amzn.to/3lkkyr1 with status code 202
Failed to fetch the url: https://amzn.to/3xzyida with status code 202
Failed to fetch the url: https://pubmed.ncbi.nlm.nih.gov/22976133/ with status code 403
Failed to fetch the url: https://journals.asm.org/doi/10.1128/msystems.00031-18 with status code 403
Failed to fetch the url: https://pubmed.ncbi.nlm.nih.gov/8370132/ with status code 403
Failed to fetch the url: https://pmc.ncbi.nlm.nih.gov/articles/pmc6611658/ with status code 403
Failed to fetch the url: https://www.mayoclinicproceedings.org/article/s0025-6196(18)30538-x/fulltext with status code 403
Failed to fetch the url: http://neurohacker.com/mbg  with status code 404
Failed to fetch the url: https://www.google.com/url with status code 404
Failed to fetch the url: https://fdc.nal.usda.gov

In [22]:
print("\n=== CREATE CSV OF NODES ===")

print(f"{BLUE}[{MBG_source_name}] Creating nodes CSV...{RESET}")
create_nodes_csv(MBG_dico_links, MBG_files['nodes']) # We convert the list of links into a dictionary {url: []} to be compatible
print(f"{GREEN}Nodes CSV saved to {MBG_files['nodes']}{RESET}")

print("\n=== CREATE CSV OF EDGES ===\n")
print(f"{BLUE}[{MBG_source_name}] Creating edges CSV...{RESET}")
create_edges_csv(MBG_dico_links, MBG_files['edges']) # Export directed edges (source -> target) from the adjacency list
print(f"{GREEN}Edges CSV saved to {MBG_files['edges']}{RESET}") 


=== CREATE CSV OF NODES ===
[MBG] Creating nodes CSV...
Nodes CSV saved to data\MBG_nodes.csv

=== CREATE CSV OF EDGES ===

[MBG] Creating edges CSV...
Edges CSV saved to data\MBG_edges.csv


###### For Wikipedia

In [23]:
wiki_source_name = "wiki"
wiki_start_point = wiki_seed_pages

wiki_files = get_filenames(wiki_source_name) # Build all standardized output paths for this source 

In [24]:
print("\n=== CRAWLING ===")

print(f"{BLUE}[{wiki_source_name}] Starting the crawling...{RESET}")    
wiki_crawled_links, wiki_dico_links = crawl_wiki_seed_links(wiki_start_point)
print(f"{GREEN}-> Found {len(wiki_crawled_links)} links{RESET}")


=== CRAWLING ===
[wiki] Starting the crawling...
Failed to fetch the url: https://en.wikipedia.org/wiki/Lifestyle_trends_and_media with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Self-care with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Healthy_diet with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Travel with status code 403
-> Found 853 links


In [25]:
print("\n=== SCRAPING ===")

print(f"{BLUE}[{wiki_source_name}] Starting the scraping...{RESET}")
get_html_content((wiki_crawled_links), wiki_files['raw']) # Fetch HTML for each crawled URL and save it as a CSV (url, html)
print(f"{GREEN}Raw HTML data saved to {wiki_files['raw']}{RESET}")   


=== SCRAPING ===
[wiki] Starting the scraping...
Failed to fetch the url: https://en.wikipedia.org/wiki/Environmentalist with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Waistcoat with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Made_to_measure with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/The_Root_(magazine) with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Gymnasium_(ancient_Greece) with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Solar_distillation with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Hot_flashes with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Ugg_boot with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Self-care with status code 403
Failed to fetch the url: https://en.wikipedia.org/wiki/Industrial_agriculture with status code 403
Failed to fetch the url: ht

In [26]:
print("\n=== CREATE CSV OF NODES ===")

print(f"{BLUE}[{wiki_source_name}] Creating nodes CSV...{RESET}")
create_nodes_csv(wiki_dico_links, wiki_files['nodes']) # We convert the list of links into a dictionary {url: []} to be compatible
print(f"{GREEN}Nodes CSV saved to {wiki_files['nodes']}{RESET}")

print("\n=== CREATE CSV OF EDGES ===\n")
print(f"{BLUE}[{wiki_source_name}] Creating edges CSV...{RESET}")
create_edges_csv(wiki_dico_links, wiki_files['edges']) # Export directed edges (source -> target) from the adjacency list
print(f"{GREEN}Edges CSV saved to {wiki_files['edges']}{RESET}") 


=== CREATE CSV OF NODES ===
[wiki] Creating nodes CSV...
Nodes CSV saved to data\wiki_nodes.csv

=== CREATE CSV OF EDGES ===

[wiki] Creating edges CSV...
Edges CSV saved to data\wiki_edges.csv


## **Data Collection of Feedspot**

## Collection of keywords

In [28]:
main_topic_words_csv = pd.read_csv("data/main_topic_words_filtered.csv")
main_topic_words_str = main_topic_words_csv.iloc[0, 0]
main_topic_words = ast.literal_eval(main_topic_words_str)
print(main_topic_words)
print(len(main_topic_words))

['accessory', 'acute', 'addendum', 'advantageous', 'affirmative', 'affix', 'affixed', 'affixing', 'aflutter', 'allay', 'allayed', 'allaying', 'allays', 'alleviate', 'alleviates', 'alzheimer', 'alzheimered', 'alzheimering', 'alzheimers', 'anxiety', 'anxietyed', 'anxietying', 'anxietys', 'anxious', 'anxioused', 'anxiousing', 'anxiousness', 'anxiouss', 'appeasement', 'append', 'appended', 'appending', 'appends', 'apprehensive', 'association', 'associations', 'attentive', 'awareness', 'awarenessed', 'awarenessing', 'awarenesss', 'benefits', 'benefitsed', 'benefitsing', 'benefitss', 'bold', 'bolded', 'bolding', 'boundaries', 'boundariesed', 'boundariesing', 'boundariess', 'breathed', 'breathing', 'breathinged', 'breathinging', 'breathings', 'breathwork', 'breathworked', 'breathworking', 'breathworks', 'buoyancy', 'burnout', 'burnouted', 'burnouting', 'burnouts', 'calming', 'calminged', 'calminging', 'calmings', 'capitulum', 'carbohydrate', 'carbohydrates', 'carping', 'censorious', 'chakra',

## Definitions implementation

In [29]:
def filter_links(links, main_topic_words=None):
    if main_topic_words is None:  # If no list of required keywords is given then we create an empty one
        main_topic_words = []
        
    filtered = []
    
    for link in links:
        l_lower = link.lower()  # Transformation of capital letter into lower case letter
        
        if main_topic_words and not any(word.lower() in l_lower for word in main_topic_words):  # The function skip the url if no required keywords are in the url
            continue
        filtered.append(link) # If the url past all the filter, it is added to the list of links
    
    return filtered

In [30]:
def extract_feedspot_seed_pages(url, limit=60):
    
    soup = to_soup(url)
    links_blogs = set()

    if not soup:
        print(f"{RED}Error: Could not fetch main blogs page: {url}{RESET}")
        return set()

    blogs = soup.find_all('a', href=True) # and tag.get('class') and 'wb-ba' in tag.get('class') and any('ext' in c for c in tag.get('class')))

    for link in blogs:
        href = link.get('href') #if link.name == 'a' else link.text.strip()  if href and "http" in href and "bloggers.feedspot.com" not in href:
        
        if not href.startswith('https?'):
            href = urljoin(url, href)

        if urlparse(href).netloc != urlparse(url).netloc:
            if any(filter in href for filter in filters_noise):
                continue

            if "mindbodygreen.com" in href:
                 continue
            
            if href and "http" in href and "feedspot" not in href :
                links_blogs.add(href)

            if len(links_blogs) >= limit: 
                break
    
    return sorted(links_blogs)

In [31]:
def extract_links_breadth_first_feedspot(url, max_levels=1, main_topic_words=main_topic_words): 
    queue = deque([(url, 0)])
    visited_links = [url]  
    last_level_links = []
    dico = {}

    while queue:
        current_url, current_level = queue.popleft()

        if current_level == max_levels:
            if current_url not in last_level_links:
                last_level_links.append(current_url)
            continue
        elif current_level > max_levels:
            continue
    
        soup = to_soup(current_url)
        
        if not soup:
            continue

        dico.setdefault(current_url, [])

        content_area = soup.body 
        
        if not content_area:
            continue
        
        links = sorted(content_area.find_all('a', href=True), key=lambda x: x.get('href'))
        
        for item in links :
            href = item.get('href')

            full_url = urljoin(current_url, href)
            full_url = full_url.split('?')[0].split('#')[0]
            full_url = full_url.lower()

            if main_topic_words and not any(word.lower() in full_url for word in main_topic_words):
                continue
            
            if any(filter in full_url for filter in filters_noise):
                continue

            dico[current_url].append(full_url)

            if full_url not in visited_links:
                visited_links.append(full_url)
                queue.append((full_url, current_level + 1))
                    
    return visited_links, last_level_links, dico

In [32]:
def crawl_feedspots_seed_links(urls, main_topic_words=None):
    if main_topic_words is None:
        main_topic_words = []

    topic_links = []
    seen_links = set()
    global_dico = {}

    for url in list(urls):                                                              
        print("Crawling the url:", url)

        # Definition of our parameters for Feedspots
        MAX_LINKS_TO_DFS = 10
        MAX_BREADTH_LEVEL = 1
        MAX_DEPTH = 1
        MAX_LINKS_PER_PAGE = 15

        # Crawl en largeur
        breadth_links, last_level_links, dico = extract_links_breadth_first_feedspot(url, max_levels=MAX_BREADTH_LEVEL)

        # Crawl en profondeur à partir des liens du dernier niveau
        depth_links = extract_links_depth_from_list(max_links_to_dfs=MAX_LINKS_TO_DFS,
                                                start_links=last_level_links, 
                                                visited_links=breadth_links.copy(), 
                                                dico=dico, max_depth=MAX_DEPTH, 
                                                max_links_per_page=MAX_LINKS_PER_PAGE)
        
        for k, v in dico.items(): 
            global_dico.setdefault(k, []) 
            global_dico[k].extend(v)

        # Retourne tous les liens uniques sous forme de liste (déjà fait pour le BFS mais pas pour le DFS)
        all_links = set(breadth_links) | set(depth_links)
        link_filtered = filter_links(list(all_links), main_topic_words=main_topic_words)
        
        for link in link_filtered :
            
            if link not in seen_links :
                topic_links.append(link)
                seen_links.add(link)

    return topic_links, global_dico

## Execution

In [33]:
blogs_source_name = "blogs"
blogs_start_point = "https://bloggers.feedspot.com/women_lifestyle_blogs/?_src=bloggers_directory"

blogs_files = get_filenames(blogs_source_name) # Build all standardized output paths for this source 

In [34]:
print("\n=== CRAWLING ===")

print(f"{BLUE}[{blogs_source_name}] Starting the crawling...{RESET}")   
seed_blogs = extract_feedspot_seed_pages(blogs_start_point, limit=60)
print(f"{BLUE}[{blogs_source_name}] Links to crawl: {RESET}",len(seed_blogs))  
blogs_crawled_links, blogs_dico_links = crawl_feedspots_seed_links(list(seed_blogs), main_topic_words=main_topic_words)
print(f"{GREEN}-> Found {len(blogs_crawled_links)} links{RESET}")


=== CRAWLING ===
[blogs] Starting the crawling...
[blogs] Links to crawl:  60
Crawling the url: https://40plusstyle.com/
Crawling the url: https://9to5chic.com/
Crawling the url: https://afrocosmopolitan.com/
Crawling the url: https://awellstyledlife.com/
Crawling the url: https://bestofthislife.com/
Crawling the url: https://bevcooks.com/
Crawling the url: https://camillestyles.com/
Crawling the url: https://caphillstyle.com/
Failed to fetch the url: https://www.wsj.com/articles/dear-j-crew-what-happened-to-us-we-used-to-be-so-close-1432661954 with status code 401
Crawling the url: https://chachingqueen.com/
Crawling the url: https://coloradomountainmom.com/
Crawling the url: https://cupofjo.com/
Crawling the url: https://dianaelizabethblog.com/
Crawling the url: https://freshmommyblog.com/
Crawling the url: https://helloadamsfamily.com/
Crawling the url: https://hellohappinessblog.com/
Crawling the url: https://hisugarplum.com/
Crawling the url: https://inspirationsandcelebrations.n

In [38]:
df_seeds = pd.DataFrame({"seed_blogs": [list(seed_blogs)]})
df_seeds.to_csv("data/feedspot_seed_blogs.csv", index=False)

In [35]:
print("\n=== SCRAPING ===")

print(f"{BLUE}[{blogs_source_name}] Starting the scraping...{RESET}")
get_html_content((blogs_crawled_links), blogs_files['raw']) # Fetch HTML for each crawled URL and save it as a CSV (url, html)
print(f"{GREEN}Raw HTML data saved to {blogs_files['raw']}{RESET}")    


=== SCRAPING ===
[blogs] Starting the scraping...
Failed to fetch the url: https://amzlink.to/az0jlomfycdcw with status code 404
Failed to fetch the url: https://amzlink.to/az039vxhb4bcl with status code 404
Failed to fetch the url: https://bestofthislife.com/2016/02/easy-weekend-brunch-table-setting.html/easy-weekend-brunch-table-setting with status code 404
Failed to fetch the url: https://www.wsj.com/articles/dear-j-crew-what-happened-to-us-we-used-to-be-so-close-1432661954 with status code 401
Failed to fetch the url: https://plus.qvc.com/series/the-ultimate-gift-wrapping-challenge/vmlkzw9fvmlkzw9tzxjpzxmkzdy1njkxzda4ymjmztmwmdg5yzczzgfina== with status code 404
Failed to fetch the url: https://amzlink.to/az0pa7gb5t1q6 with status code 404
Failed to fetch the url: https://amzlink.to/az0ay9d8uvifg with status code 404
Failed to fetch the url: https://amzlink.to/az0wac1nlvop0 with status code 404
Failed to fetch the url: https://amzlink.to/az0msgfqfcl5y with status code 404
Failed t

In [36]:
print("\n=== CREATE CSV OF NODES ===")

print(f"{BLUE}[{blogs_source_name}] Creating nodes CSV...{RESET}")
create_nodes_csv(blogs_dico_links, blogs_files['nodes']) # We convert the list of links into a dictionary {url: []} to be compatible
print(f"{GREEN}Nodes CSV saved to {blogs_files['nodes']}{RESET}")

print("\n=== CREATE CSV OF EDGES ===\n")
print(f"{BLUE}[{blogs_source_name}] Creating edges CSV...{RESET}")
create_edges_csv(blogs_dico_links, blogs_files['edges']) # Export directed edges (source -> target) from the adjacency list
print(f"{GREEN}Edges CSV saved to {blogs_files['edges']}{RESET}") 


=== CREATE CSV OF NODES ===
[blogs] Creating nodes CSV...
Nodes CSV saved to data\blogs_nodes.csv

=== CREATE CSV OF EDGES ===

[blogs] Creating edges CSV...
Edges CSV saved to data\blogs_edges.csv
